In [1]:
import os
os.chdir('../../') # it should be ../../

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50 , InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input

In [15]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ClassicalModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    model_name: str
    num_classes:int
    num_train_epochs: int
    batch_size: int
    loss: str
    optimizer: str
    metrics: str
    img_size: [int]
    
    

In [16]:
from HybridQCCNN.constants import *
from HybridQCCNN.utils.common import read_yaml, create_directories, get_size

In [10]:
from HybridQCCNN.components.data_transformation import DataTransformation
from HybridQCCNN.entity import DataIngetionConfig

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        
    def get_model_trainer_config(self)-> ClassicalModelTrainerConfig:
        
        config=self.config.classical_model_config
        params=self.params.ClassicalTrainingArguments
        
        create_directories([config.root_dir])
        model_trainer_config = ClassicalModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            batch_size = params.train_batch_size,
            model_name= config.model_name,
            num_classes=config.num_classes,
            optimizer= params.optimizer,
            loss=params.loss,
            metrics= params.metrics,
            img_size= config.img_size

        )
        return model_trainer_config  
    
    
    def get_data_ingetion_config(self)-> DataIngetionConfig:
        
        config=self.config.data_ingetion
        
        create_directories([config.root_dir])
        data_ingetion_config=DataIngetionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingetion_config  

In [19]:
class ClassicalModelTrainer:
    def __init__(self,
                 config:ClassicalModelTrainerConfig):
        self.config=config
        
        
    def build_model(self):
        if self.config.model_name=='ResNet50':
            base_model=ResNet50(weights='imagenet')
        elif self.config.model_name=='InceptionV3':
            base_model=InceptionV3(weights='imagenet')
            
            
        rescale = preprocessing.Rescaling(1./255)
        
        
        inputs = Input(shape=(self.config.img_size[0],self.config.img_size[1],3))
        x = rescale(inputs)
        x=base_model(x)
        x = layers.Dense(256, activation='relu')(x)
        predictions = layers.Dense(self.config.num_classes, activation='softmax')(x)
        
        self.model = models.Model(inputs=inputs, outputs=predictions)

    def train(self, train_data):
        model=self.model
        model.compile(optimizer=self.config.optimizer, loss=self.config.loss, metrics=[self.config.metrics])
        
        model.fit(train_data , epochs=self.config.num_train_epochs,steps_per_epoch=len(train_data))
        
    

In [20]:
try:
    configer=ConfigurationManager()
    config=configer.get_model_trainer_config()
    trainer= ClassicalModelTrainer(config=config)
    trainer.build_model()

    data_ingetion_config=configer.get_data_ingetion_config()
    train_data=DataTransformation(data_ingetion_config).data_loader()
    trainer.train(train_data=train_data)
    
except Exception as e:
    raise e
    

[2024-01-09 10:51:51,492 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-01-09 10:51:51,494 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-01-09 10:51:51,497 : INFO : common : created directory at: model/]
[2024-01-09 10:51:53,524 : INFO : common : created directory at: dataset/data_ingestion]
Found 13711 files belonging to 23 classes.
Using 10969 files for training.
  4/343 [..............................] - ETA: 1:00:48 - loss: 3.1342 - accuracy: 0.1016  

KeyboardInterrupt: 